## Estatísticas de Meios de Pagamento - v1

Começo em 20241/1 até hoje, com todas as tabelas disponíveis da API

In [2]:
url = "https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/odata/MeiosdePagamentosMensalDA(AnoMes=@AnoMes)?@AnoMes='202401'&$top=100&$format=json&$select=AnoMes,quantidadePix,valorPix,quantidadeTED,valorTED,quantidadeTEC,valorTEC,quantidadeCheque,valorCheque,quantidadeBoleto,valorBoleto,quantidadeDOC,valorDOC"

Importar bibliotecas

In [3]:
import requests  #possibilita integração com serviços web, solicitações HTTP consumir dados de APIs
import json  #manipular arquivo json

Criar uma função para obter os dados:

In [4]:

def requisicao_api(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:      #sucesso na requisição
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('resultado.json', 'w', encoding='utf-8') as arquivo:   #salvar arquivo json
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)


Chamar função passando o link gerado pela interface da API

In [5]:
requisicao_api(url)

Status Code: 200


In [6]:
with open("resultado.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

# Extrai apenas o conteúdo da chave "value"
registros = dados["value"]


In [7]:
print(registros)

[{'AnoMes': '202509', 'quantidadePix': 6853600.54, 'valorPix': 3180234.46, 'quantidadeTED': 67787.76, 'valorTED': 3935730.15, 'quantidadeTEC': 0.0, 'valorTEC': 0.0, 'quantidadeCheque': 8401.46, 'valorCheque': 36257.82, 'quantidadeBoleto': 349079.13, 'valorBoleto': 569213.66, 'quantidadeDOC': 0.0, 'valorDOC': 0.0}, {'AnoMes': '202508', 'quantidadePix': 6975792.66, 'valorPix': 3035727.36, 'quantidadeTED': 64506.6, 'valorTED': 3700999.64, 'quantidadeTEC': 0.0, 'valorTEC': 0.0, 'quantidadeCheque': 7764.15, 'valorCheque': 34009.59, 'quantidadeBoleto': 328959.8, 'valorBoleto': 522435.28, 'quantidadeDOC': 0.0, 'valorDOC': 0.0}, {'AnoMes': '202507', 'quantidadePix': 6681904.26, 'valorPix': 3058009.49, 'quantidadeTED': 67192.89, 'valorTED': 3915166.65, 'quantidadeTEC': 0.0, 'valorTEC': 0.0, 'quantidadeCheque': 8507.87, 'valorCheque': 35797.81, 'quantidadeBoleto': 353819.55, 'valorBoleto': 564569.38, 'quantidadeDOC': 0.0, 'valorDOC': 0.0}, {'AnoMes': '202506', 'quantidadePix': 6463470.81, 'valor

###Objetivo: Processamento de dados com Spark

Instalando Java e Pyspark:

In [8]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-11-jdk -qq > /dev/null
!pip install -q pyspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Definir variáveis de ambiente para especificar onde o Spark está instalado:

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

Importar Bibliotecas e iniciar seção:

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round

# ---- Criar sessão Spark ----
spark = SparkSession.builder \
    .appName("BCB_API_PySpark") \
    .master("local[*]") \
    .getOrCreate()


Início ao processamento de dados:

In [11]:
df = spark.createDataFrame(registros)

# Mostra o esquema e os dados
df.printSchema()

root
 |-- AnoMes: string (nullable = true)
 |-- quantidadeBoleto: double (nullable = true)
 |-- quantidadeCheque: double (nullable = true)
 |-- quantidadeDOC: double (nullable = true)
 |-- quantidadePix: double (nullable = true)
 |-- quantidadeTEC: double (nullable = true)
 |-- quantidadeTED: double (nullable = true)
 |-- valorBoleto: double (nullable = true)
 |-- valorCheque: double (nullable = true)
 |-- valorDOC: double (nullable = true)
 |-- valorPix: double (nullable = true)
 |-- valorTEC: double (nullable = true)
 |-- valorTED: double (nullable = true)



In [12]:
df.show(5, truncate=False)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|valorPix  |valorTEC|valorTED  |
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|202509|349079.13       |8401.46         |0.0          |6853600.54   |0.0          |67787.76     |569213.66  |36257.82   |0.0     |3180234.46|0.0     |3935730.15|
|202508|328959.8        |7764.15         |0.0          |6975792.66   |0.0          |64506.6      |522435.28  |34009.59   |0.0     |3035727.36|0.0     |3700999.64|
|202507|353819.55       |8507.87         |0.0          |6681904.26   |0.0          |67192.89     |564569.38  |35797.81   |0.0     |3058009.49|0.0     |3915166.65|
|202506|338127.37     

**Transformações:** Criam novos DataFrames sem processar imediatamente os dados

In [13]:
from pyspark.sql import functions as F

In [14]:
# 1. Selecionar colunas específicas
df_select = df.select("quantidadeBoleto", "valorBoleto")
df_select.show(10)

+----------------+-----------+
|quantidadeBoleto|valorBoleto|
+----------------+-----------+
|       349079.13|  569213.66|
|        328959.8|  522435.28|
|       353819.55|  564569.38|
|       338127.37|  523439.24|
|       340006.42|  532979.61|
|       341624.79|  539022.34|
|        340788.0|  516117.79|
|        324481.7|  488180.37|
|       338952.59|   519244.8|
|       361042.81|  566693.06|
+----------------+-----------+
only showing top 10 rows



In [15]:
# 2. Criar novas colunas
df_novo = df.withColumn("valor_total", F.col("valorBoleto") + F.col("valorCheque"))
df_novo.show(5)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+-----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|valor_total|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+-----------+
|202509|       349079.13|         8401.46|          0.0|   6853600.54|          0.0|     67787.76|  569213.66|   36257.82|     0.0|3180234.46|     0.0|3935730.15|  605471.48|
|202508|        328959.8|         7764.15|          0.0|   6975792.66|          0.0|      64506.6|  522435.28|   34009.59|     0.0|3035727.36|     0.0|3700999.64|  556444.87|
|202507|       353819.55|         8507.87|          0.0|   6681904.26|          0.0|     67192.89|  564569.38|   35797.81|   

In [16]:
# 3. Filtrar registros (condição)
df_filtrado = df.filter(F.col("quantidadeCheque") > 8000)
df_filtrado.show(10)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|202509|       349079.13|         8401.46|          0.0|   6853600.54|          0.0|     67787.76|  569213.66|   36257.82|     0.0|3180234.46|     0.0|3935730.15|
|202507|       353819.55|         8507.87|          0.0|   6681904.26|          0.0|     67192.89|  564569.38|   35797.81|     0.0|3058009.49|     0.0|3915166.65|
|202506|       338127.37|         8217.12|          0.0|   6463470.81|          0.0|     62477.43|  523439.24|   34182.86|     0.0|2868306.07|     0.0|3742911.28|
|202505|       340006.

In [17]:
# 4. Ordenar valores
df_ordenado = df.orderBy(F.col("valorCheque").desc())
df_ordenado.show(10)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|202404|       361633.28|        11179.22|          0.0|   4900465.64|          0.0|     70647.15|  524619.69|    43295.4|     0.0| 2062614.3|     0.0|3652606.48|
|202401|       363550.32|        11919.12|       600.76|   4350076.85|       387.18|     70343.25|  516702.44|   41615.89|  616.58|1804382.92|  815.94|3337344.85|
|202407|       366484.15|         10942.8|          0.0|   5392531.01|          0.0|     69657.15|  546660.21|   41504.78|     0.0|2337719.13|     0.0|3828948.82|
|202405|       349180.

In [18]:
# 5. Selecionar e renomear colunas
df_renomeado = df.select(
    F.col("quantidadeBoleto").alias("qtd_boleto"),
    F.col("valorBoleto").alias("vlr_boleto")
)
df_renomeado.show(5)

+----------+----------+
|qtd_boleto|vlr_boleto|
+----------+----------+
| 349079.13| 569213.66|
|  328959.8| 522435.28|
| 353819.55| 564569.38|
| 338127.37| 523439.24|
| 340006.42| 532979.61|
+----------+----------+
only showing top 5 rows



In [26]:
# 6. Criar uma coluna categórica com base em condição
df_categoria = df.withColumn(
    "categoria_boleto",
    F.when(F.col("quantidadeBoleto") > 340000, "Alto Volume")
     .otherwise("Baixo Volume")
)
df_categoria.show(5)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+----------------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|categoria_boleto|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+----------------+
|202509|       349079.13|         8401.46|          0.0|   6853600.54|          0.0|     67787.76|  569213.66|   36257.82|     0.0|3180234.46|     0.0|3935730.15|     Alto Volume|
|202508|        328959.8|         7764.15|          0.0|   6975792.66|          0.0|      64506.6|  522435.28|   34009.59|     0.0|3035727.36|     0.0|3700999.64|    Baixo Volume|
|202507|       353819.55|         8507.87|          0.0|   6681904.26|          0.0|     67192.89|  

**Ações**: Executam o plano de transformação e retornam resultados

In [27]:
# 1. Mostrar registros
df.show(5)

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|202509|       349079.13|         8401.46|          0.0|   6853600.54|          0.0|     67787.76|  569213.66|   36257.82|     0.0|3180234.46|     0.0|3935730.15|
|202508|        328959.8|         7764.15|          0.0|   6975792.66|          0.0|      64506.6|  522435.28|   34009.59|     0.0|3035727.36|     0.0|3700999.64|
|202507|       353819.55|         8507.87|          0.0|   6681904.26|          0.0|     67192.89|  564569.38|   35797.81|     0.0|3058009.49|     0.0|3915166.65|
|202506|       338127.

In [20]:
# 2. Contar número total de registros
print("Total de registros:", df.count())

Total de registros: 21


In [28]:
# 3. Descrever estatísticas básicas
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|            AnoMes|  quantidadeBoleto|  quantidadeCheque|     quantidadeDOC|    quantidadePix|     quantidadeTEC|    quantidadeTED|       valorBoleto|       valorCheque|          valorDOC|          valorPix|          valorTEC|          valorTED|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|                21|                21|                21|                21|               21|                21|               21|                21|                21|                21|                21|                21|

In [29]:
# 4. Calcular médias de colunas
df.select(
    F.mean("quantidadeBoleto").alias("média_Boleto"),
    F.mean("valorTED").alias("média_ted")
).show()

+-----------------+------------------+
|     média_Boleto|         média_ted|
+-----------------+------------------+
|344643.2476190476|3659719.1871428574|
+-----------------+------------------+



In [23]:
# 5. Obter o registro com maior valorBoleto
df.orderBy(F.col("valorBoleto").desc()).limit(6).show()

+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|AnoMes|quantidadeBoleto|quantidadeCheque|quantidadeDOC|quantidadePix|quantidadeTEC|quantidadeTED|valorBoleto|valorCheque|valorDOC|  valorPix|valorTEC|  valorTED|
+------+----------------+----------------+-------------+-------------+-------------+-------------+-----------+-----------+--------+----------+--------+----------+
|202509|       349079.13|         8401.46|          0.0|   6853600.54|          0.0|     67787.76|  569213.66|   36257.82|     0.0|3180234.46|     0.0|3935730.15|
|202507|       353819.55|         8507.87|          0.0|   6681904.26|          0.0|     67192.89|  564569.38|   35797.81|     0.0|3058009.49|     0.0|3915166.65|
|202508|        328959.8|         7764.15|          0.0|   6975792.66|          0.0|      64506.6|  522435.28|   34009.59|     0.0|3035727.36|     0.0|3700999.64|
|202506|       338127.

In [30]:
# 6. Calcular soma total por uma categoria criada
df_categoria.groupBy("categoria_boleto").agg(
    F.sum("valorPix").alias("soma_pix"),
    F.sum("valorTED").alias("soma_ted")
).show()

+----------------+-------------+-------------+
|categoria_boleto|     soma_pix|     soma_ted|
+----------------+-------------+-------------+
|    Baixo Volume|1.900609362E7|2.787430407E7|
|     Alto Volume|3.250760821E7|4.897979886E7|
+----------------+-------------+-------------+



------

---